# Using Databases and SQL

### Create a database and table

In [1]:
import sqlite3

conn = sqlite3.connect('music.sqlite')
cur = conn.cursor()

cur.execute('DROP TABLE IF EXISTS Tracks')
cur.execute('CREATE TABLE Tracks (title TEXT, plays INTEGER)')

### Insert two rows of values

In [2]:
cur.execute('INSERT INTO Tracks (title, plays) VALUES (?, ?)', 
    ('Thunderstruck', 20))
cur.execute('INSERT INTO Tracks (title, plays) VALUES (?, ?)', 
    ('My Way', 15))
conn.commit()

### Show the tows

In [3]:
print('Tracks:')
cur.execute('SELECT title, plays FROM Tracks')
for row in cur:
     print(row)


Tracks:
('Thunderstruck', 20)
('My Way', 15)


### Delete all rows where plays < 18

In [4]:
cur.execute('DELETE FROM Tracks WHERE plays < 18')
conn.commit()

cur.execute('SELECT * FROM Tracks')
for row in cur:
    print(row)

cur.close()

('Thunderstruck', 20)


### Crawl friends of Twitter user and store them in DB

In [1]:
from urllib.request import urlopen
import urllib.error
import twurl
import json
import sqlite3
import ssl

TWITTER_URL = 'https://api.twitter.com/1.1/friends/list.json'

conn = sqlite3.connect('spider.sqlite')
cur = conn.cursor()

cur.execute('''
            CREATE TABLE IF NOT EXISTS Twitter
            (name TEXT, retrieved INTEGER, friends INTEGER)''')

# Ignore SSL certificate errors
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

while True:
    acct = input('Enter a Twitter account, or quit: ')
    if (acct == 'quit'): break
    if (len(acct) < 1):
        cur.execute('SELECT name FROM Twitter WHERE retrieved = 0 LIMIT 1')
        try:
            acct = cur.fetchone()[0]
        except:
            print('No unretrieved Twitter accounts found')
            continue

    url = twurl.augment(TWITTER_URL, {'screen_name': acct, 'count': '5'})
    print('Retrieving', url)
    connection = urlopen(url, context=ctx)
    data = connection.read().decode()
    headers = dict(connection.getheaders())

    print('Remaining', headers['x-rate-limit-remaining'])
    js = json.loads(data)
    # Debugging
    # print json.dumps(js, indent=4)

    cur.execute('UPDATE Twitter SET retrieved=1 WHERE name = ?', (acct, ))

    countnew = 0
    countold = 0
    for u in js['users']:
        friend = u['screen_name']
        print(friend)
        cur.execute('SELECT friends FROM Twitter WHERE name = ? LIMIT 1',
                    (friend, ))
        try:
            count = cur.fetchone()[0]
            cur.execute('UPDATE Twitter SET friends = ? WHERE name = ?',
                        (count+1, friend))
            countold = countold + 1
        except:
            cur.execute('''INSERT INTO Twitter (name, retrieved, friends)
                        VALUES (?, 0, 1)''', (friend, ))
            countnew = countnew + 1
    print('New accounts=', countnew, ' revisited=', countold)
    conn.commit()

cur.close()

Enter a Twitter account, or quit: drsris
Retrieving https://api.twitter.com/1.1/friends/list.json?oauth_consumer_key=ObG0rFIZON5mHUcXHbwhBIB5A&oauth_timestamp=1567238858&oauth_nonce=32784092&oauth_version=1.0&screen_name=drsris&count=5&oauth_token=836931359880835072-srBegAOc9dC3oWWoJFKrvlBzng8Trk6&oauth_signature_method=HMAC-SHA1&oauth_signature=ilpV8SGc5XmszoZxnpKvtdW8Dy4%3D
Remaining 5
suneet4037
MatthieuLefe
gaurav_singal
shreyanse081
LeadingindiaAI
New accounts= 0  revisited= 5
Enter a Twitter account, or quit: 
Retrieving https://api.twitter.com/1.1/friends/list.json?oauth_consumer_key=ObG0rFIZON5mHUcXHbwhBIB5A&oauth_timestamp=1567238872&oauth_nonce=21807744&oauth_version=1.0&screen_name=LeadingindiaAI&count=5&oauth_token=836931359880835072-srBegAOc9dC3oWWoJFKrvlBzng8Trk6&oauth_signature_method=HMAC-SHA1&oauth_signature=7OAd6tBxM2cXjewC8G%2Bn6fqgrMQ%3D
Remaining 4
drsris
CSE_Bennett
deep108
DeepMindAI
IBMWatson
New accounts= 4  revisited= 1
Enter a Twitter account, or quit: 
Retri